<a href="https://colab.research.google.com/github/Abhirrami/sdc/blob/main/TRANSLATOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers gradio sentencepiece

from transformers import MarianMTModel, MarianTokenizer
import gradio as gr

# Mapping between display names and model language codes
lang_options = {
    "English": "en",
    "French": "fr",
    "German": "de",
    "Spanish": "es",
    "Russian": "ru",
    "Hindi": "hi",
    "Italian": "it",
    "Chinese": "zh",
    "Japanese": "ja"
}

# Function to load translation model dynamically
def load_translation_model(src_lang_code, tgt_lang_code):
    model_name = f"Helsinki-NLP/opus-mt-{src_lang_code}-{tgt_lang_code}"
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

# Translation function
def translate_text(text, src_lang, tgt_lang):
    if src_lang == tgt_lang:
        return text

    src_code = lang_options.get(src_lang)
    tgt_code = lang_options.get(tgt_lang)

    try:
        tokenizer, model = load_translation_model(src_code, tgt_code)
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        translated_tokens = model.generate(**inputs, max_length=512)
        translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        return translated_text
    except Exception as e:
        return f"Translation failed: {str(e)}"

# Gradio Interface
gr.Interface(
    fn=translate_text,
    inputs=[
        gr.Textbox(label="Enter Text to Translate"),
        gr.Dropdown(choices=list(lang_options.keys()), label="From Language", value="English"),
        gr.Dropdown(choices=list(lang_options.keys()), label="To Language", value="French")
    ],
    outputs=gr.Textbox(label="Translated Text"),
    title="Google Translator (Hugging Face + Gradio)",
    description="Translate between multiple languages using Hugging Face MarianMT models."
).launch()
